# All the parameters that need to be changed

In [1]:
# Louisiana
state_ab = "la"

## Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [35]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest19_data = "./{}{}_vest_19/{}_vest_19.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2022/HB1_Congress_221ES.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2022/SB1_Senate_221ES.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2022/HB14_House_221ES.shp".format(data_folder, state_ab)

## Parameters that needs to be manually checked

### base vest data
start_col = 5\
vest_base_data = vest20\
year = '20'

### district data
district column name of cong_df, send, hdist when calling add_dist()

# Program starts

In [3]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

maup.progress.enabled = True

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [6]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [7]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [8]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [9]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [10]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [11]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [12]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [13]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
               'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
               'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
               'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [14]:
population_df.rename(columns=rename_dict, inplace = True)

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['NH_WHITE'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['NH_BLACK'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['NH_AMIN'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['NH_ASIAN'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['NH_NHPI'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['NH_OTHER'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['NH_2MORE'], 1)

# Read the base vest data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [16]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

### Check if vest 20 can be used as base

In [17]:
vest20 = gpd.read_file(vest20_data)

In [18]:
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|████████████████████████████████████████| 3753/3753 [00:09<00:00, 395.31it/s]


There are 1 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 3798/3798 [00:01<00:00, 2173.42it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|████████████████████████████████████████| 3753/3753 [00:07<00:00, 475.42it/s]


### If it is true for maup doctor, we will use it as the base vest data.
Check where the election column starts, this should be the same for all vest data in that state

In [19]:
vest20.columns

Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAME', 'G20PRERTRU', 'G20PREDBID',
       'G20PRELJOR', 'G20PREOWES', 'G20PREOOTH', 'G20USSRCAS', 'G20USSRMUR',
       'G20USSDPER', 'G20USSDEDW', 'G20USSDPIE', 'G20USSDKNI', 'G20USSDWEN',
       'G20USSLSIG', 'G20USSIMEN', 'G20USSOPRI', 'G20USSOJOH', 'G20USSNBIL',
       'G20USSNBOU', 'G20USSNMON', 'G20USSNDAR', 'geometry'],
      dtype='object')

## Parameters that need to be checked

In [20]:
start_col = 4
vest_base_data = vest20
year = '20'

In [21]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [22]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|█████████████████████████████████████████| 3753/3753 [01:02<00:00, 59.95it/s]


In [23]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [24]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [25]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

### Check if the population agrees

In [26]:
check_population(population_df, vest_base)

population agrees


# Add more vest data

In [28]:
vest19 = gpd.read_file(vest19_data)

In [29]:
vest19.columns

Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAME', 'G19GOVRRIS', 'G19GOVRABR',
       'G19GOVRLAN', 'G19GOVDEDW', 'G19GOVDDAN', 'G19GOVILAN', 'G19LTGRNUN',
       'G19LTGDJON', 'G19SOSRARD', 'G19SOSRKEN', 'G19SOSRSMI', 'G19SOSDCOL',
       'G19ATGRLAN', 'G19ATGDJAC', 'G19TRERSCH', 'G19TREDEDW', 'G19TRENKEN',
       'G19AGRRSTR', 'G19AGRRZAU', 'G19AGRDGR1', 'G19AGRDGR2', 'G19AGRDWIL',
       'G19INSRDON', 'G19INSRTEM', 'R19GOVRRIS', 'R19GOVDEDW', 'R19SOSRARD',
       'R19SOSDCOL', 'geometry'],
      dtype='object')

In [30]:
# check the result here
election_df = add_vest(vest19, election_df, '19', population_df, start_col)

100%|████████████████████████████████████████| 3752/3752 [00:08<00:00, 451.20it/s]


There are 1 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 3797/3797 [00:01<00:00, 2289.29it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 3753/3753 [00:59<00:00, 62.84it/s]


population agrees


In [27]:
vest18 = gpd.read_file(vest18_data)
vest18.columns

Index(['STATEFP10', 'COUNTYFP10', 'VTDST10', 'NAME10', 'G18SOSRARD',
       'G18SOSREDM', 'G18SOSRSTO', 'G18SOSRKEN', 'G18SOSRCRO', 'G18SOSRCLO',
       'G18SOSDCOL', 'G18SOSDFRE', 'G18SOSNMOR', 'R18SOSRARD', 'R18SOSDCOL',
       'geometry'],
      dtype='object')

In [31]:
# check the result here
election_df = add_vest(vest18, election_df, '18', population_df, start_col)

100%|████████████████████████████████████████| 3710/3710 [00:08<00:00, 458.18it/s]


There are 1 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 3748/3748 [00:01<00:00, 2280.97it/s]


Resolving overlaps...
Filling gaps...


Gaps to simplify: 100%|█████████████████████████████| 1/1 [00:00<00:00,  2.02it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 3753/3753 [00:59<00:00, 62.89it/s]


population agrees


In [32]:
vest16 = gpd.read_file(vest16_data)
vest16.columns

Index(['STATEFP10', 'COUNTYFP10', 'VTDST10', 'NAME10', 'G16PRERTRU',
       'G16PREDCLI', 'G16PRELJOH', 'G16PREGSTE', 'G16PREOMCM', 'G16PRECCAS',
       'G16PREOOTH', 'G16USSRKEN', 'G16USSRBOU', 'G16USSRFLE', 'G16USSRMAN',
       'G16USSRDUK', 'G16USSRCRA', 'G16USSRCAO', 'G16USSRMAR', 'G16USSRPAT',
       'G16USSDCAM', 'G16USSDFAY', 'G16USSDEDW', 'G16USSDLAN', 'G16USSDPEL',
       'G16USSDWIL', 'G16USSDMEN', 'G16USSLCLE', 'G16USSLGIL', 'G16USSOLAN',
       'G16USSOTAY', 'G16USSNBIL', 'G16USSNHEB', 'G16USSNMAR', 'G16USSNWEL',
       'R16USSRKEN', 'R16USSDCAM', 'geometry'],
      dtype='object')

In [33]:
election_df = add_vest(vest16, election_df, '16', population_df, start_col)

100%|████████████████████████████████████████| 3704/3704 [00:08<00:00, 437.18it/s]


There are 1 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 3742/3742 [00:01<00:00, 2224.16it/s]


Resolving overlaps...
Filling gaps...


Gaps to simplify: 100%|█████████████████████████████| 1/1 [00:00<00:00,  2.65it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 3753/3753 [01:01<00:00, 60.66it/s]


population agrees


## Add the district data

In [36]:
cong_df = gpd.read_file(cd_data).to_crs('EPSG:4269')
send = gpd.read_file(send_data).to_crs('EPSG:4269')
hdist = gpd.read_file(hdist_data).to_crs('EPSG:4269')

In [37]:
cong_df.head()

,ID,AREA,OBJECTID,DISTRICT_I,NAME,NUM_MEMBER,IDEAL_POP,IS_LOCKED,SHAPE_AREA,SHAPE_LEN,geometry
0,5,15208.31440,5,5,District 5,1,776292,0,3.746101,19.820423,"POLYGON ((-92.06893 33.00799, -92.06840 33.007..."
1,3,8612.33105,3,3,District 3,1,776292,0,2.082512,9.691517,"POLYGON ((-91.75621 30.49813, -91.75580 30.498..."
2,4,13676.68750,4,4,District 4,1,776292,0,3.369344,16.654527,"POLYGON ((-92.06893 33.00799, -92.06894 33.007..."
3,1,9001.86621,1,1,District 1,1,776292,0,2.170494,13.215370,"POLYGON ((-90.54688 30.43161, -90.54679 30.431..."
4,6,4452.85351,6,6,District 6,1,776292,1,1.077026,13.590078,"POLYGON ((-91.70134 30.49775, -91.67922 30.497..."


In [38]:
election_df = add_district(cong_df, "CD", election_df, "DISTRICT_I")

100%|███████████████████████████████████████████████| 6/6 [00:12<00:00,  2.15s/it]


In [39]:
send.head()

,ID,AREA,OBJECTID,DISTRICT_I,NAME,NUM_MEMBER,IDEAL_POP,IS_LOCKED,SHAPE_AREA,SHAPE_LEN,geometry
0,21,3313.98730,33,33,District 33,1,119429,0,0.825928,8.256837,"POLYGON ((-93.52099 33.01860, -93.51473 33.018..."
1,20,4222.72119,32,32,District 32,1,119429,0,1.039192,11.102783,"POLYGON ((-91.51474 31.60941, -91.51301 31.606..."
2,12,2129.16577,26,26,District 26,1,119429,0,0.514721,4.514334,"POLYGON ((-92.04562 29.50581, -92.04734 29.506..."
3,18,1241.25817,29,29,District 29,1,119429,0,0.306078,9.155427,"POLYGON ((-92.93895 32.52407, -92.93886 32.524..."
4,15,1704.75268,35,35,District 35,1,119429,0,0.421254,6.352017,"POLYGON ((-92.67721 32.54723, -92.67640 32.547..."


In [41]:
election_df = add_district(send, "SEND", election_df, "DISTRICT_I")

100%|█████████████████████████████████████████████| 39/39 [00:00<00:00, 42.27it/s]


There are 1 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|████████████████████████████████████████████| 43/43 [00:00<00:00, 419.90it/s]


Resolving overlaps...
1 gaps will remain unfilled, because they either are not simply connected or exceed the area threshold.
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 39/39 [00:00<00:00, 43.24it/s]


There are 1 holes.


Exception: maup.doctor failed

In [40]:
hdist.head()

,ID,AREA,OBJECTID,DISTRICT_I,NAME,NUM_MEMBER,IDEAL_POP,IS_LOCKED,SHAPE_AREA,SHAPE_LEN,geometry
0,17,1269.450560,12,12,District 12,1,44359,0,0.316265,2.972525,"POLYGON ((-92.72474 33.01439, -92.72365 33.014..."
1,7,3456.005120,47,47,District 47,1,44359,0,0.834558,6.079176,"POLYGON ((-92.04562 29.50581, -92.04734 29.506..."
2,20,231.143982,15,15,District 15,1,44359,0,0.057390,1.363273,"POLYGON ((-92.41551 32.49561, -92.41546 32.503..."
3,45,2135.672360,20,20,District 20,1,44359,0,0.527201,7.543825,"POLYGON ((-92.32225 32.32963, -92.32201 32.329..."
4,31,1065.128660,28,28,District 28,1,44359,0,0.260345,3.158888,"POLYGON ((-92.00803 31.32581, -92.00621 31.325..."


In [42]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICT_I")

100%|███████████████████████████████████████████| 105/105 [00:10<00:00,  9.80it/s]


In [44]:
election_df.columns

Index(['2MOREVAP', 'AGR19D', 'AGR19R', 'AMINVAP', 'ASIANVAP', 'ATG19D',
       'ATG19R', 'BVAP', 'COUNTYFP', 'GOV19D', 'GOV19O', 'GOV19R', 'HISP',
       'HVAP', 'H_2MORE', 'H_AMIN', 'H_ASIAN', 'H_BLACK', 'H_NHPI', 'H_OTHER',
       'H_WHITE', 'INS19R', 'LTG19D', 'LTG19R', 'NAME', 'NHPIVAP', 'NH_2MORE',
       'NH_AMIN', 'NH_ASIAN', 'NH_BLACK', 'NH_NHPI', 'NH_OTHER', 'NH_WHITE',
       'OTHERVAP', 'PRE16D', 'PRE16O', 'PRE16R', 'PRE20D', 'PRE20O', 'PRE20R',
       'SOS18D', 'SOS18O', 'SOS18R', 'SOS19D', 'SOS19R', 'STATEFP', 'TOTPOP',
       'TRE19D', 'TRE19O', 'TRE19R', 'USS16D', 'USS16O', 'USS16R', 'USS20D',
       'USS20O', 'USS20R', 'VAP', 'VTDST', 'WVAP', 'geometry', 'CD', 'HDIST'],
      dtype='object')

### Put the base precinct year after the precinct information column

In [45]:
base_columns = {}
if 'COUNTYFP' + year not in election_df.columns:
    base_columns = {
        'STATEFP':'STATEFP'+year,
        'COUNTYFP':'COUNTYFP'+year,
        'VTDST':'VTDST'+year,
        'GEOID':'GEOID'+year,
        'NAME':'NAME'+year}
election_df.rename(columns=base_columns, inplace = True)

In [48]:
# reorder the columns
fixed_columns = [
    'STATEFP'+year,
    'COUNTYFP'+year,
    'VTDST'+year,
    # 'GEOID'+year,
    'NAME'+year,
    'CD',
    # 'SEND', failed
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [49]:
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))